In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from testing import Tester

/home/wpx/miniconda3/envs/stlit/lib/python3.13/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


# Test Cases
- For all 12 cases, 3 tests for sanity check of the optimization result.
    1. Sum of individual PPA provision matches total PPA provision.
    2. Matches the target CFE level: total PPA provision / total demand >= CFE target
    3. Consistency of optimal price: sum of individual costs matches the final PPA price computated.

- Most using default parameters
    - Penalty: using sample spot price, pen_factor = 3, pen_cap = 500
    - New gen: using sample 1MW wind, LCOE = 30, PAP PPA price = 20
    - Existing/New bat: LCOS = 120, RTE = 0.9, Max_charging_cycle = 1, capacity = 100MWh, charging_rate = 25 MW
        - sub case for battery: LCOS = 50 for some cases with penalty


- **Warning**: 
    1. *A lot of hard coding*
    2. Cases involving **New Battery** and **New Asset** often fail the some test cases due to rounding error, check the print statement for details.

### Case 0 No Pen No Bat

In [ ]:
# Case 0
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_0_0.75_optimization_output.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_0_0.75_Price.txt', dtype='str', delimiter=','))[21:28])

lcoe = [20, 23, 25, 30, 35]

tester = Tester(0, 0.75, 5, 3, lcoe, opt_output, opt_price)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 1 No Pen No Bat New Asset

In [ ]:
# Case 0
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_1_0.85_optimization_output.csv')
opt_price = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_1_0.85_ppa_price.txt', dtype='str')[0][4][:7])
new_asset_cap = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_1_0.85_ppa_price.txt', dtype='str')[1][4][:7])
# print(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_1_0.85_ppa_price.txt', dtype='str')[0][4])
# opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_1_0.85_ppa_price.txt', dtype='str', delimiter=','))[21:28])


lcoe = [20, 23, 25, 30, 35]
new_asset_lcoe = 30

tester = Tester(0, 0.85, 6, 3, lcoe, opt_output, opt_price, 
                new_asset_lcoe = new_asset_lcoe, new_asset_pap_price = 20)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 2 No Pen New Bat

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_2_0.85_optimization_output.csv')
opt_price = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_2_0.85_ppa_price.txt', dtype='str')[0][4][:6])

bat_capacity = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_2_0.85_ppa_price.txt', dtype='str')[1][4][:6])
bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_2_0.85_bat_output.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

case_type = 2
matching_lvl = 0.85
lcoe = [20, 23, 25, 30, 35]
no_gen = 5
no_cus = 3

bat_params = [120, bat_capacity, 0.9, 25, 1, 4]

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                bat_change = bat_change, bat_params = bat_params)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()


[PASS] Total RE Provision Matched with Aggregated Quantity
[ERROR] Total Energy Provision NOT Matched with Demand at Target CFE%
Cur level 0.84428
[ERROR] Optimal price NOT Matched
computed: 43.5529 In result: 43.553
re_cost 26704208.97968602
bat_cost 15872681.999999998
mkt cost 0.0
res excess rev 0
total prov 977590.9385275298


### Case 3 No Pen New Bat New Asset

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_3_0.95_optimization_output.csv')
opt_price = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_3_0.95_ppa_price.txt', dtype='str')[0][4][:6])

new_asset_cap = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_3_0.95_ppa_price.txt', dtype='str')[1][4][:7])

bat_capacity = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_3_0.95_ppa_price.txt', dtype='str')[2][4][:6])
bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_3_0.95_bat_output.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()


print('new asset cap', new_asset_cap)
print('new bat cap', bat_capacity)
print('')

case_type = 3
matching_lvl = 0.95
lcoe = [20, 23, 25, 30, 35]
no_gen = 6
no_cus = 3

bat_params = [120, bat_capacity, 0.9, 25, 1, 4]
new_asset_lcoe = 30

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                bat_change = bat_change, bat_params = bat_params,
                new_asset_lcoe = new_asset_lcoe, new_asset_pap_price = 20)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()


new asset cap 228.264
new bat cap 88.345

[ERROR] Total RE Provision NOT Matched with Aggregated Quantity
Number of unmatched cases: 846

Showing First 10 Lines of Mismatched Values
Row 40 | Row sums 125.4 | Total gen value 125.41
Row 63 | Row sums 122.63 | Total gen value 122.64
Row 65 | Row sums 121.58 | Total gen value 121.57
Row 82 | Row sums 125.69 | Total gen value 125.68
Row 83 | Row sums 126.33 | Total gen value 126.34
Row 93 | Row sums 120.24 | Total gen value 120.25
Row 94 | Row sums 115.98 | Total gen value 115.97
Row 244 | Row sums 83.64 | Total gen value 83.65
Row 285 | Row sums 110.15 | Total gen value 110.14
Row 288 | Row sums 109.5 | Total gen value 109.51
Row 290 | Row sums 96.99 | Total gen value 97.0

[PASS] Total Energy Provision Matched with Demand at Target CFE%
[ERROR] Optimal price NOT Matched
computed: 32.7554 In result: 32.755
re_cost 33791613.223729834
bat_cost 3869511.0
mkt cost 0.0
res excess rev 1872533.2654708656
total prov 1092601.6371778203


### Case 4 No Pen Ex Bat

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_4_0.75_optimization_output.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_4_0.75_ppa_price.txt', dtype='str', delimiter=','))[21:28])


bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_4_0.75_bat_ouput.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()


lcoe = [20, 23, 25, 30, 35]
bat_params = [120, 100, 0.9, 25, 1, 4]

tester = Tester(4, 0.75, 5, 4, lcoe, opt_output, opt_price, bat_change = bat_change, bat_params = bat_params)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 5 No Pen Ex Bat New Asset

In [ ]:

opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_5_0.75_optimization_output.csv')
opt_price = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_5_0.75_ppa_price.txt', dtype='str')[0][4][:7])
new_asset_cap = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_5_0.75_ppa_price.txt', dtype='str')[1][4][:7])


bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_5_0.75_bat_output.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

case_type = 5
matching_lvl = 0.75
lcoe = [20, 23, 25, 30, 35]
no_gen = 6
no_cus = 4

lcoe = [20, 23, 25, 30, 35]
bat_params = [120, 100, 0.9, 25, 1, 4]
new_asset_lcoe = 30

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                bat_change = bat_change, bat_params = bat_params,  
                new_asset_lcoe = new_asset_lcoe, new_asset_pap_price = 30)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 6 Pen No Bat

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_6_0.9_optimization_output.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_6_0.9_ppa_price.txt', dtype='str', delimiter=','))[21:28])

case_type = 6
matching_lvl = 0.9
lcoe = [20, 23, 25, 30, 35]
no_gen = 5
no_cus = 4

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500


tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = 500)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 7 Pen New Asset

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_7_0.9_optimization_output.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_7_0.9_ppa_price.txt', dtype='str', delimiter=','))[21:28])

case_type = 7
matching_lvl = 0.9
lcoe = [20, 23, 25, 30, 35]
no_gen = 6
no_cus = 4

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500

new_asset_lcoe = 30
new_asset_ppa_price = 20


tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = 500,
                new_asset_lcoe = new_asset_lcoe, new_asset_pap_price = new_asset_ppa_price)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[ERROR] Total RE Provision NOT Matched with Aggregated Quantity
Number of unmatched cases: 15

Showing First 10 Lines of Mismatched Values
Row 316 | Row sums 94.63 | Total gen value 94.62
Row 317 | Row sums 94.83 | Total gen value 94.82
Row 604 | Row sums 91.35 | Total gen value 91.36
Row 700 | Row sums 99.61 | Total gen value 99.6
Row 941 | Row sums 109.95 | Total gen value 109.96
Row 963 | Row sums 114.11 | Total gen value 114.12
Row 1083 | Row sums 106.28 | Total gen value 106.29
Row 1155 | Row sums 99.92 | Total gen value 99.91
Row 1156 | Row sums 94.33 | Total gen value 94.34
Row 1301 | Row sums 99.4 | Total gen value 99.39
Row 1468 | Row sums 94.62 | Total gen value 94.63

[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 8 Pen New Bat

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_optimization_output.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_ppa_price.txt', dtype='str', delimiter=','))[23:28])

bat_capacity = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_ppa_price.txt', dtype='str')[1][4][:2])
bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_bat_output.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

case_type = 8
matching_lvl = 0.95
lcoe = [20, 23, 25, 30, 35]
no_gen = 5
no_cus = 3

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500

print(spot_price[:10])

bat_params = [120, bat_capacity, 0.9, 25, 1, 4]

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                bat_change = bat_change, bat_params = bat_params,
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = pen_cap)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[20.575 19.622 18.447 19.323 19.203 17.848 18.609 20.838 20.251 19.583]
[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[ERROR] Optimal price NOT Matched
computed: 30.6213 In result: 30.62
re_cost 23077508.265809633
bat_cost 0.0
mkt cost 10379319.79801407
res excess rev 0
total prov 1092601.6371778224


In [ ]:
# Case with battery, by putting lcos = 50

opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_optimization_output_50_bat.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_ppa_price_50_bat.txt', dtype='str', delimiter=','))[23:28])

bat_capacity = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_ppa_price_50_bat.txt', dtype='str')[1][4][:6])
bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_8_0.95_bat_output_50_bat.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

print('Bat capacity', bat_capacity)
case_type = 8
matching_lvl = 0.95
lcoe = [20, 23, 25, 30, 35]
no_gen = 5
no_cus = 3

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500

print(spot_price[:10])

bat_params = [50, bat_capacity, 0.9, 25, 1, 4]

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                bat_change = bat_change, bat_params = bat_params,
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = pen_cap)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

'''
RE cost 23270704.348512013
MKT cost 9741082.728850057
total_prov 1092601.6371778247
'''

Bat capacity 22.882
[20.575 19.622 18.447 19.323 19.203 17.848 18.609 20.838 20.251 19.583]
[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[ERROR] Optimal price NOT Matched
computed: 30.5961 In result: 30.59
re_cost 23270704.348512013
bat_cost 417596.50000000006
mkt cost 9741082.728850057
res excess rev 0
total prov 1092601.6371778212


'\nRE cost 23270704.348512013\nMKT cost 9741082.728850057\ntotal_prov 1092601.6371778247\n'

### Case 9 Pen New Bat New Asset

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_9_1_optimization_output_50_bat.csv')
opt_price = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_9_1_ppa_price_50_bat.txt', dtype='str')[0][4][:6])

new_asset_cap = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_9_1_ppa_price_50_bat.txt', dtype='str')[1][4][:7])

bat_capacity = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_9_1_ppa_price_50_bat.txt', dtype='str')[2][4][:7])
bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_9_1_bat_output_50_bat.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500

print('new asset cap', new_asset_cap)
print('new bat cap', bat_capacity)
print('')

case_type = 9
matching_lvl = 1
lcoe = [20, 23, 25, 30, 35]
no_gen = 6
no_cus = 3

bat_params = [50, bat_capacity, 0.9, 25, 1, 4]
new_asset_lcoe = 30

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                bat_change = bat_change, bat_params = bat_params,
                new_asset_lcoe = new_asset_lcoe, new_asset_pap_price = 20,
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = pen_cap)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

'''
Case 8
RE cost 28245053.979420554
MKT cost 7034504.146243073
Bat cost 660675.5233809971
MKT rev 41037.02422575044
total_prov 1150106.9865029736
'''

new asset cap 131.162
new bat cap 36.2014

[ERROR] Total RE Provision NOT Matched with Aggregated Quantity
Number of unmatched cases: 228

Showing First 10 Lines of Mismatched Values
Row 330 | Row sums 122.14 | Total gen value 122.15
Row 606 | Row sums 93.23 | Total gen value 93.24
Row 620 | Row sums 116.21 | Total gen value 116.22
Row 621 | Row sums 110.37 | Total gen value 110.38
Row 629 | Row sums 43.5 | Total gen value 0.0
Row 767 | Row sums 124.53 | Total gen value 124.52
Row 828 | Row sums 61.16 | Total gen value 0.0
Row 957 | Row sums 127.88 | Total gen value 127.89
Row 962 | Row sums 116.31 | Total gen value 116.32
Row 1083 | Row sums 102.28 | Total gen value 102.29
Row 1106 | Row sums 104.89 | Total gen value 104.9

[PASS] Total Energy Provision Matched with Demand at Target CFE%
[ERROR] Optimal price NOT Matched
computed: 31.2138 In result: 31.213
re_cost 28245053.979420558
bat_cost 660675.5499999999
mkt cost 7034504.146243073
res excess rev 41037.02422575046
total prov 11501

'\nCase 8\nRE cost 28245053.979420554\nMKT cost 7034504.146243073\nBat cost 660675.5233809971\nMKT rev 41037.02422575044\ntotal_prov 1150106.9865029736\n'

### Case 10 Pen Ex Bat

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_10_0.9_optimization_output.csv')
opt_price = float(str(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_10_0.9_ppa_price.txt', dtype='str', delimiter=','))[21:28])

bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_10_0.9_bat_output.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500

case_type = 10
matching_lvl = 0.9
lcoe = [20, 23, 25, 30, 35]
no_gen = 5
no_cus = 3

bat_params = [120, 100, 0.9, 25, 1, 4]

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = 500,
                bat_change = bat_change, bat_params = bat_params)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[PASS] Total RE Provision Matched with Aggregated Quantity
[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched


### Case 11 Pen Ex Bat New Asset

In [ ]:
opt_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_11_1_optimization_output.csv')
opt_price = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_11_1_ppa_price.txt', dtype='str')[0][4][:7])
new_asset_cap = float(np.loadtxt('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_11_1_ppa_price.txt', dtype='str')[1][4][:7])

bat_output = pd.read_csv('/home/wpx/TP-REC/PPA_Aggregator_Github/outputs_testing/case_11_1_bat_output.csv')
bat_change = bat_output['Change in Energy (MWh)'].to_numpy()

spot_price = pd.read_excel('/home/wpx/TP-REC/PPA_Aggregator_v3_Solar_Simu/input/input/spot_2023.xlsx').to_numpy().flatten()
pen_factor = 3
pen_cap = 500

case_type = 11
matching_lvl = 1
lcoe = [20, 23, 25, 30, 35]
no_gen = 6
no_cus = 3

bat_params = [120, 100, 0.9, 25, 1, 4]
new_asset_lcoe = 30
new_asset_ppa_price = 20

tester = Tester(case_type, matching_lvl, no_gen, no_cus, lcoe, opt_output, opt_price, 
                spot_price = spot_price, pen_factor = pen_factor, pen_cap = 500,
                bat_change = bat_change, bat_params = bat_params,
                new_asset_lcoe = new_asset_lcoe, new_asset_pap_price = new_asset_ppa_price)

tester.test_total_re_gen()
tester.test_matching_level()
tester.test_optimal_price()

[ERROR] Total RE Provision NOT Matched with Aggregated Quantity
Number of unmatched cases: 24

Showing First 10 Lines of Mismatched Values
Row 316 | Row sums 90.63 | Total gen value 90.62
Row 604 | Row sums 85.51 | Total gen value 85.5
Row 1160 | Row sums 113.55 | Total gen value 113.56
Row 1161 | Row sums 119.07 | Total gen value 119.08
Row 1466 | Row sums 98.78 | Total gen value 98.79
Row 1467 | Row sums 93.23 | Total gen value 93.24
Row 1469 | Row sums 92.65 | Total gen value 92.64
Row 1670 | Row sums 118.16 | Total gen value 118.15
Row 1679 | Row sums 110.89 | Total gen value 110.9
Row 1699 | Row sums 117.92 | Total gen value 117.93
Row 1709 | Row sums -0.01 | Total gen value 0.0

[PASS] Total Energy Provision Matched with Demand at Target CFE%
[PASS] Optimal Price Matched
